In [1]:
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import math
from scipy.spatial import ConvexHull, convex_hull_plot_2d
import imageio
import numpy as np
import warnings
import torch
from torch import nn
import torch.optim as opt

In [2]:
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model

model = load_model('model_full_fold.h5')

In [3]:
def get_tracking_and_plays(filepath):
    """
    Return a dataframe containing cleaned tracking data joined with the plays data
    """
    tracking = pd.read_csv(filepath)
    plays = pd.read_csv("plays.csv")
    tracking = tracking[tracking['playId'].isin(tracking[tracking['event'] != 'fumble']['playId'].unique())]
    plays = plays[plays['playNullifiedByPenalty'] == 'N']

    tracking.loc[tracking['playDirection'] == 'left', 'x'] = 120 - tracking.loc[tracking['playDirection'] == 'left', 'x']
    tracking.loc[tracking['playDirection'] == 'left', 'y'] = (160/3) - tracking.loc[tracking['playDirection'] == 'left', 'y']
    tracking.loc[tracking['playDirection'] == 'left', 'dir'] += 180
    tracking.loc[tracking['dir'] > 360, 'dir'] -= 360
    tracking.loc[tracking['playDirection'] == 'left', 'o'] += 180
    tracking.loc[tracking['o'] > 360, 'o'] -= 360

    tracking_with_plays = tracking.merge(plays, on=['gameId', 'playId'], how='left')

    tracking_with_plays['is_on_offense'] = tracking_with_plays['club'] == tracking_with_plays['possessionTeam']
    tracking_with_plays['is_on_defence'] = tracking_with_plays['club'] == tracking_with_plays['defensiveTeam']
    tracking_with_plays['is_ballcarrier'] = tracking_with_plays['ballCarrierId'] == tracking_with_plays['nflId']

    end_frame = tracking_with_plays[tracking_with_plays['event'].isin(['tackle', 'out_of_bounds'])].groupby('playId')['frameId'].min().reset_index()
    end_frame.rename(columns={'frameId': 'frameId_end'}, inplace=True)

    start_frame = tracking_with_plays[tracking_with_plays['event'].isin(['run', 'lateral', 'run_pass_option', 'handoff', 'pass_arrived'])].groupby('playId')['frameId'].min().reset_index()
    start_frame.rename(columns={'frameId': 'frameId_start'}, inplace=True)

    tracking_with_plays = tracking_with_plays.merge(end_frame, on='playId', how='left')
    tracking_with_plays = tracking_with_plays.merge(start_frame, on='playId', how='left')

    tracking_with_plays = tracking_with_plays[(tracking_with_plays['frameId'] <= tracking_with_plays['frameId_end']) &
                                              (tracking_with_plays['frameId'] >= tracking_with_plays['frameId_start'])]

    return tracking_with_plays

In [4]:
tracking_with_plays = get_tracking_and_plays('tracking_week_3.csv')

,gameId,playId,nflId,displayName,frameId,time,jerseyNumber,club,playDirection,x,...,expectedPointsAdded,foulName1,foulName2,foulNFLId1,foulNFLId2,is_on_offense,is_on_defence,is_ballcarrier,frameId_end,frameId_start


In [5]:
def create_feature_tensor(feature_df):
    """
    Convert the input frame_df to a 4D tensor.
        - The first dimension is the frame
        - The second dimension is the index of the current player
        - The third dimension is the index of the relative player
    """
    tensor_shape = (feature_df.groupby(['gameId', 'playId', 'frameId']).ngroups, 10, 11, 10)
    input_tensor = np.zeros(tensor_shape)
    cur_count = 0
    for game_id, game_group in feature_df.groupby('gameId'):
        for play_id, play_group in game_group.groupby('playId'):
            for frame_id, frame_group in play_group.groupby('frameId'):
                offense = frame_group[(frame_group['is_on_offense']) & (~frame_group['is_ballcarrier'])]
                defence = frame_group[frame_group['is_on_defence']]
                ballcarrier = frame_group[frame_group['is_ballcarrier']]
                ballcarrier_sx = ballcarrier.s * np.cos(np.deg2rad(ballcarrier.dir))
                ballcarrier_sy = ballcarrier.s * np.sin(np.deg2rad(ballcarrier.dir))
                for i, def_player in enumerate(defence.itertuples()):
                    def_player_sx = def_player.s * np.cos(np.deg2rad(def_player.dir))
                    def_player_sy = def_player.s * np.sin(np.deg2rad(def_player.dir))
                    for j, off_player in enumerate(offense.itertuples()):
                        off_player_sx = off_player.s * np.cos(np.deg2rad(off_player.dir))
                        off_player_sy = off_player.s * np.sin(np.deg2rad(off_player.dir))
                        input_tensor[cur_count, 0, i, j] = off_player.x - def_player.x
                        input_tensor[cur_count, 1, i, j] = off_player.y - def_player.y
                        input_tensor[cur_count, 2, i, j] = def_player_sx
                        input_tensor[cur_count, 3, i, j] = def_player_sy
                        input_tensor[cur_count, 4, i, j] = def_player_sx - ballcarrier_sx
                        input_tensor[cur_count, 5, i, j] = def_player_sy - ballcarrier_sy
                        input_tensor[cur_count, 6, i, j] = def_player.x - ballcarrier.x
                        input_tensor[cur_count, 7, i, j] = def_player.y - ballcarrier.y
                        input_tensor[cur_count, 8, i, j] = off_player_sx - def_player_sx
                        input_tensor[cur_count, 9, i, j] = off_player_sy - def_player_sy
                cur_count += 1
    return input_tensor

Counterfactual Analysis

In [6]:
import numpy as np

def change_distance(df, displayName, amount):
    df['sx'] = df['s'] * np.cos(np.deg2rad(df['dir']))
    df['sy'] = df['s'] * np.sin(np.deg2rad(df['dir']))

    filtered_df = df[df['displayName'] == displayName].copy()
    filtered_df['px'] = filtered_df['x'].shift(1)
    filtered_df['pxs'] = filtered_df['sx'].shift(1)
    filtered_df['py'] = filtered_df['y'].shift(1)
    filtered_df['psy'] = filtered_df['sy'].shift(1)

    for index, row in filtered_df.iterrows():
        if pd.isna(row['px']) or pd.isna(row['pxs']) or pd.isna(row['py']) or pd.isna(row['psy']):
            continue
        
        # print(df.at[index, 'x'])
        df.at[index, 'sx'] = row['pxs']
        df.at[index, 'sy'] = row['psy']
        df.at[index, 'x'] = row['px'] + amount * row['pxs']
        df.at[index, 'y'] = row['py'] + amount * row['psy']
        # print(df.at[index, 'x'])
    return df

In [7]:
def compare_yard_prediction(tracking_with_plays, displayName, gameId, playId, amount):
  play_info = tracking_with_plays.loc[(tracking_with_plays['playId'] == playId) & (tracking_with_plays['gameId'] == gameId)]
  original_prediction = model.predict(create_feature_tensor(play_info))
  changed_play_info = change_distance(play_info, displayName, amount)
  changed_prediction = model.predict(create_feature_tensor(changed_play_info))
  prediction_difference = changed_prediction - original_prediction
  results = pd.DataFrame({
        'Original Prediction': original_prediction.flatten(),
        'Changed Prediction': changed_prediction.flatten(),
        'Prediction Difference': prediction_difference.flatten()
    })
  print(1)
  return results

In [8]:
compare_yard_prediction(tracking_with_plays, 'Terrell Edmunds', 2022092200, 601, 0.1)

/var/folders/pq/qr32tkk90n7361gslj40w8j4tyz0cb/T/ipykernel_82194/1026185271.py:29: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  input_tensor[cur_count, 4, i, j] = def_player_sx - ballcarrier_sx
/var/folders/pq/qr32tkk90n7361gslj40w8j4tyz0cb/T/ipykernel_82194/1026185271.py:30: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  input_tensor[cur_count, 5, i, j] = def_player_sy - ballcarrier_sy
/var/folders/pq/qr32tkk90n7361gslj40w8j4tyz0cb/T/ipykernel_82194/1026185271.py:31: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  input_tensor[cur_count, 6, i, j] = def_player.x - ballcarrier.x
/var/folders/pq/qr32tkk90n7361gslj40w8j4tyz0cb/T/ipykernel_82194/1026185271.py:32: FutureWarning: Calling float on a sin

2/2 [==============================] - 0s 6ms/step


/var/folders/pq/qr32tkk90n7361gslj40w8j4tyz0cb/T/ipykernel_82194/207683585.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sx'] = df['s'] * np.cos(np.deg2rad(df['dir']))
/var/folders/pq/qr32tkk90n7361gslj40w8j4tyz0cb/T/ipykernel_82194/207683585.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sy'] = df['s'] * np.sin(np.deg2rad(df['dir']))
/var/folders/pq/qr32tkk90n7361gslj40w8j4tyz0cb/T/ipykernel_82194/1026185271.py:29: FutureWarning: Calling float on a single element Series is deprecated and

2/2 [==============================] - 0s 3ms/step
1


,Original Prediction,Changed Prediction,Prediction Difference
0,1.265527,1.265527,0.000000
1,1.294765,1.294067,-0.000697
2,1.333917,1.333606,-0.000312
3,1.363687,1.363737,0.000050
4,1.381868,1.382601,0.000733
5,1.395878,1.398013,0.002134
6,1.364179,1.367338,0.003159
7,1.324557,1.331004,0.006447
8,1.357734,1.363408,0.005674
9,1.364234,1.374025,0.009791


In [9]:
plays=pd.read_csv('plays.csv')
games=pd.read_csv('games.csv')
tracking1 = get_tracking_and_plays('tracking_week_1.csv')
tracking2 = get_tracking_and_plays('tracking_week_2.csv')
tracking3 = get_tracking_and_plays('tracking_week_3.csv')
tracking4 = get_tracking_and_plays('tracking_week_4.csv')
tracking5 = get_tracking_and_plays('tracking_week_5.csv')
tracking6 = get_tracking_and_plays('tracking_week_6.csv')
tracking7 = get_tracking_and_plays('tracking_week_7.csv')
tracking8 = get_tracking_and_plays('tracking_week_8.csv')
tracking9 = get_tracking_and_plays('tracking_week_9.csv')

In [10]:
def get_tracking(week_num):
    if week_num==1:
        return tracking1
    elif week_num==2:
        return tracking2
    elif week_num==3:
        return tracking3
    elif week_num==4:
        return tracking4
    elif week_num==5:
        return tracking5
    elif week_num==6:
        return tracking6
    elif week_num==7:
        return tracking7
    elif week_num==8:
        return tracking8
    else:
        return tracking9

In [179]:
warnings.filterwarnings('ignore')
distinct_plays=plays.sort_values(by=['gameId','playId'])[['gameId', 'playId']]
results_dict={}

counter=0
for i, row in distinct_plays.iterrows():
    week_number=int(games.loc[games['gameId']==row['gameId']]['week'])
    tracking=get_tracking(week_number)
    players=tracking.loc[(tracking['gameId']==row['gameId']) & (tracking['playId']==row['playId']) & (tracking['is_on_defence']==True)]['displayName'].unique()
    for j in players:
        final_df=compare_yard_prediction(tracking, j, int(row['gameId']), int(row['playId']), 0.1)
        cumulative_impact=sum(final_df['Prediction Difference'])
        if j in results_dict:
            results_dict[j]+=cumulative_impact
        else:
            results_dict[j]=cumulative_impact
    counter+=1
    if counter>10:
        break

1/1 [==============================] - 0s 10ms/step
1
17
1/1 [==============================] - 0s 9ms/step
1
17
1/1 [==============================] - 0s 11ms/step
1
17
1/1 [==============================] - 0s 12ms/step
1
17
1/1 [==============================] - 0s 12ms/step
1
17
1/1 [==============================] - 0s 10ms/step


ValueError: setting an array element with a sequence.

Difference between Expected Yards and Realized Yards at Start of Play

In [11]:
def get_prediction_difference(tracking_with_plays, gameId, playId):
  play_info = tracking_with_plays.loc[(tracking_with_plays['playId'] == playId) & (tracking_with_plays['gameId'] == gameId)]
  first_coord_football=float(play_info[(play_info['frameId']==play_info['frameId_start']) & (play_info['displayName']=='football')]['x'])
  last_coord_football=float(play_info[(play_info['frameId']==play_info['frameId_end']) & (play_info['displayName']=='football')]['x'])
  original_prediction = model.predict(create_feature_tensor(play_info))
  actual_yards=last_coord_football-first_coord_football
  ball_carrier=play_info.iloc[0]['ballCarrierDisplayName']
  defense_name=play_info.iloc[0]['defensiveTeam']
  return actual_yards-original_prediction[0][0], ball_carrier, defense_name

In [12]:
get_prediction_difference(tracking1, 2022090800, 818)

/var/folders/pq/qr32tkk90n7361gslj40w8j4tyz0cb/T/ipykernel_82194/2488794438.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  first_coord_football=float(play_info[(play_info['frameId']==play_info['frameId_start']) & (play_info['displayName']=='football')]['x'])
/var/folders/pq/qr32tkk90n7361gslj40w8j4tyz0cb/T/ipykernel_82194/2488794438.py:4: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  last_coord_football=float(play_info[(play_info['frameId']==play_info['frameId_end']) & (play_info['displayName']=='football')]['x'])
/var/folders/pq/qr32tkk90n7361gslj40w8j4tyz0cb/T/ipykernel_82194/1026185271.py:29: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  input_tensor[cur_count, 4, i, j] = def_player_sx 

1/1 [==============================] - 0s 15ms/step


(1.9375271797180602, 'Darrell Henderson', 'BUF')

In [30]:
warnings.filterwarnings('ignore')
distinct_plays=plays.sort_values(by=['gameId','playId'])[['gameId', 'playId']]
team_results={}

games=games.sort_values(by='week')

counter=0
for i, row in distinct_plays.iterrows():
    week_number=int(games.loc[games['gameId']==row['gameId']]['week'])
    tracking=get_tracking(week_number)
    try:
        tuple_vals=get_prediction_difference(tracking, row['gameId'], row['playId'])
        if tuple_vals[1] in team_results:
         team_results[tuple_vals[1]]+=tuple_vals[0]
        else:
            team_results[tuple_vals[1]]=tuple_vals[0]
        if tuple_vals[2] in team_results:
            team_results[tuple_vals[2]]-=tuple_vals[0]
        else:
            team_results[tuple_vals[2]]=tuple_vals[0]*-1
    except:
        print(row['gameId'], row['playId'])
    counter+=1

team_results

1/1 [==============================] - 0s 9ms/step
2022100201 3542
1/1 [==============================] - 0s 10ms/step
2022100202 709
1/1 [==============================] - 0s 9ms/step
2022100202 881
2/2 [==============================] - 0s 2ms/step
2022100202 1009
2/2 [==============================] - 0s 2ms/step
2022100202 1293
2/2 [==============================] - 0s 2ms/step
2022100202 1369
1/1 [==============================] - 0s 10ms/step
2022100202 1433
1/1 [==============================] - 0s 9ms/step
2022100202 2019
1/1 [==============================] - 0s 9ms/step
2022100202 2373
1/1 [==============================] - 0s 9ms/step
2022100202 2574
2/2 [==============================] - 0s 2ms/step
2022100202 3268
2/2 [==============================] - 0s 2ms/step
2022100202 3483
1/1 [==============================] - 0s 9ms/step
2022100202 3621
2/2 [==============================] - 0s 1ms/step
2022100203 2440
1/1 [==============================] - 0s 10ms/step
2022100203

{'Stefon Diggs': 76.70538434386184,
 'LA': -339.78115597367446,
 'Josh Allen': 87.92232686281224,
 'Devin Singletary': 86.04957437515299,
 'Zack Moss': 68.79053702950544,
 'Darrell Henderson': -14.729316651819747,
 'BUF': -453.1729350984132,
 'Cooper Kupp': 44.876804918051114,
 'Tyler Higbee': 64.56052201986364,
 'Ben Skowronek': 37.36804196238532,
 'Jamison Crowder': 2.415125489235052,
 'Isaiah McKenzie': 6.111850976943934,
 'Dawson Knox': -26.23320931196196,
 'Cam Akers': -79.77077853679748,
 'Gabe Davis': 11.16139400005316,
 'Brandon Powell': 40.243643820286955,
 'Allen Robinson': -20.70005300641074,
 'Matthew Stafford': -3.0891325473784974,
 'Marcus Mariota': 123.30519008636465,
 'NO': -140.67162051796657,
 'Cordarrelle Patterson': 23.24673151970007,
 'Parker Hesse': 13.075969815255043,
 'Damien Williams': -2.1463484764098766,
 'Kyle Pitts': 59.74632185697537,
 'Alvin Kamara': 29.95114758610742,
 'ATL': -244.07688891887352,
 'Jameis Winston': -3.116720199585018,
 'Olamide Zaccheaus

In [35]:
import operator
sorted_d = sorted(team_results.items(), key=operator.itemgetter(1), reverse=True)
sorted_d

[('Lamar Jackson', 439.4007861018178),
 ('Travis Etienne', 285.69930595159553),
 ('Justin Fields', 279.5212503373616),
 ('Miles Sanders', 274.7611335515975),
 ('Rhamondre Stevenson', 223.9665178358567),
 ('Khalil Herbert', 211.95844608545283),
 ('Kyler Murray', 184.53632259368902),
 ('Saquon Barkley', 182.89138776064118),
 ('Dameon Pierce', 165.26517754793426),
 ('Aaron Jones', 162.49561327695807),
 ('Justin Jefferson', 160.74112063646353),
 ('Kenyan Drake', 157.46185284853),
 ('Austin Ekeler', 151.79632234573302),
 ('Dallas Goedert', 143.32377177476965),
 ('Terry McLaurin', 137.83208358287843),
 ('Tyler Allgeier', 137.61533141136047),
 ('Christian McCaffrey', 137.3318694233894),
 ('David Montgomery', 135.41521602869113),
 ('Deebo Samuel', 133.47172492742573),
 ('Joe Mixon', 127.55844670534117),
 ('David Njoku', 127.48708564043018),
 ('Jaylen Waddle', 125.41608536243342),
 ('Marcus Mariota', 123.30519008636465),
 ('Jaylen Warren', 120.0756590068334),
 ('James Conner', 118.8274457454697

Tackle Evasion Analysis

In [13]:
def get_broken_tackles(tracking_with_plays, gameId, playId):
  play_info = tracking_with_plays.loc[(tracking_with_plays['playId'] == playId) & (tracking_with_plays['gameId'] == gameId)].reset_index()
  original_prediction = model.predict(create_feature_tensor(play_info))
  index=np.argmax(original_prediction<1)
  tackle_evasion=0
  tackle_evaded_yards=0
  ball_carrier=play_info.iloc[0]['ballCarrierDisplayName']
  defense_name=play_info.iloc[0]['defensiveTeam']
  if index>0:
    first_coord_football=float(play_info[(play_info['displayName']=='football')].iloc[index]['x'])
    last_coord_football=float(play_info[(play_info['frameId']==play_info['frameId_end']) & (play_info['displayName']=='football')]['x'])
    if (last_coord_football-first_coord_football)>5:
      tackle_evasion=1
      tackle_evaded_yards=last_coord_football-first_coord_football 
  return tackle_evasion, tackle_evaded_yards, ball_carrier, defense_name

In [26]:
warnings.filterwarnings('ignore')
distinct_plays=plays.sort_values(by=['gameId','playId'], ascending=False)[['gameId', 'playId']]
ball_carrier_results_tackles_evaded={}
ball_carrier_results_tackes_yards={}
defense_results_tackles_evaded={}
defense_results_tackles_yards={}

games=games.sort_values(by='week')

counter=0
for i, row in distinct_plays.iterrows():
    week_number=int(games.loc[games['gameId']==row['gameId']]['week'])
    tracking=get_tracking(week_number)
    try:
        tuple_vals=get_broken_tackles(tracking, row['gameId'], row['playId'])
        if tuple_vals[2] in ball_carrier_results_tackles_evaded:
            ball_carrier_results_tackles_evaded[tuple_vals[2]]+=tuple_vals[0]
            ball_carrier_results_tackes_yards[tuple_vals[2]]+=tuple_vals[1]
        else:
            ball_carrier_results_tackles_evaded[tuple_vals[2]]=tuple_vals[0]
            ball_carrier_results_tackes_yards[tuple_vals[2]]=tuple_vals[1]
        if tuple_vals[3] in defense_results_tackles_evaded:
            defense_results_tackles_evaded[tuple_vals[3]]-=tuple_vals[0]
            defense_results_tackles_yards[tuple_vals[3]]-=tuple_vals[1]
        else:
            defense_results_tackles_evaded[tuple_vals[3]]=tuple_vals[0]
            defense_results_tackles_yards[tuple_vals[3]]=tuple_vals[1]
    except:
        print(row['gameId'], row['playId'])
    counter+=1

print(ball_carrier_results_tackles_evaded)
print(ball_carrier_results_tackes_yards)
print(defense_results_tackles_evaded)
print(defense_results_tackles_yards)

1/1 [==============================] - 0s 10ms/step
2022110700 3480
2/2 [==============================] - 0s 2ms/step
2022110700 3401
1/1 [==============================] - 0s 10ms/step
2022110700 3195
1/1 [==============================] - 0s 10ms/step
2022110700 2496
1/1 [==============================] - 0s 10ms/step
2022110700 2412
2/2 [==============================] - 0s 1ms/step
2022110700 2182
1/1 [==============================] - 0s 9ms/step
2022110700 1768
2022110700 1734
1/1 [==============================] - 0s 10ms/step
2022110700 839
2/2 [==============================] - 0s 2ms/step
2022110700 700
2/2 [==============================] - 0s 2ms/step
2022110700 646
1/1 [==============================] - 0s 11ms/step
2022110700 518
1/1 [==============================] - 0s 10ms/step
2022110610 4348
2/2 [==============================] - 0s 2ms/step
2022110610 4162
1/1 [==============================] - 0s 9ms/step
2022110610 3836
1/1 [==============================] - 0s 9

In [25]:
import operator
sorted_d = sorted(ball_carrier_results_tackles_evaded.items(), key=operator.itemgetter(1), reverse=True)
sorted_d

[('Kenyan Drake', 6),
 ('Josh Jacobs', 3),
 ('Lamar Jackson', 2),
 ('Malcolm Brown', 2),
 ('Antonio Gibson', 2),
 ('Rhamondre Stevenson', 2),
 ('Khalil Herbert', 2),
 ('David Montgomery', 2),
 ('Austin Ekeler', 2),
 ('Derrick Henry', 1),
 ('Travis Kelce', 1),
 ('Chigoziem Okonkwo', 1),
 ('Leonard Fournette', 1),
 ('Cade Otton', 1),
 ("Ke'Shawn Vaughn", 1),
 ('James Conner', 1),
 ('Travis Homer', 1),
 ('Colby Parkinson', 1),
 ('Brian Robinson', 1),
 ('Armani Rogers', 1),
 ('Zach Wilson', 1),
 ('James Robinson', 1),
 ('James Cook', 1),
 ('Garrett Wilson', 1),
 ('Jonnu Smith', 1),
 ('Kendrick Bourne', 1),
 ('Christian Kirk', 1),
 ('Luke Farrell', 1),
 ('Trevor Lawrence', 1),
 ('Justin Jackson', 1),
 ('Samaje Perine', 1),
 ('Trenton Irwin', 1),
 ('Joe Mixon', 1),
 ('Justin Fields', 1),
 ('Dwayne Washington', 0),
 ('Alvin Kamara', 0),
 ("Tre'Quan Smith", 0),
 ('Chris Olave', 0),
 ('Marquez Callaway', 0),
 ('Andy Dalton', 0),
 ('Kevin White', 0),
 ('Taysom Hill', 0),
 ('Justice Hill', 0),
 (

In [152]:
tracking3.loc[(tracking3['playId'] == 419)& (tracking3['gameId']==2022092200)]['gameClock']

6388    8:37
6389    8:37
6390    8:37
6391    8:37
6392    8:37
        ... 
8133    8:37
8134    8:37
8135    8:37
8136    8:37
8137    8:37
Name: gameClock, Length: 1288, dtype: object

In [ ]:
warnings.filterwarnings('ignore')
distinct_plays=plays.sort_values(by=['gameId','playId'])[['gameId', 'playId']]
player_results_broken_tackles={}
player_results_expected_yards={}

games=games.sort_values(by='week')

counter=0
for i, row in distinct_plays.iterrows():
    week_number=int(games.loc[games['gameId']==row['gameId']]['week'])
    tracking=get_tracking(week_number)
    try:
        tuple_vals=get_prediction_difference(tracking, row['gameId'], row['playId'])
        if tuple_vals[1] in team_results:
         player_results[tuple_vals[1]]+=tuple_vals[0]
        else:
            team_results[tuple_vals[1]]=tuple_vals[0]
        if tuple_vals[2] in team_results:
            team_results[tuple_vals[2]]-=tuple_vals[0]
        else:
            team_results[tuple_vals[2]]=tuple_vals[0]*-1
    except:
        print(row['gameId'], row['playId'])
    counter+=1

team_results